# Model using Resampled train and test data

(may be invalid due to synthetic test data that may be deemed unsuitable)

### Dataset Prep

Data import, dataset cleaning, libraries.
In this section - try data augment, separating diff attacks.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from io import StringIO


from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# Import dataset
from google.colab import drive
drive.mount('/content/drive/')
df = pd.read_csv("/content/drive/MyDrive/ALLFLOWMETER_HIKARI2021.csv")
df.head(1)

In [ ]:
# Dropping inherent packet features.

df.drop(['uid','originh','originp','responh','responp'], axis=1, inplace=True)

In [ ]:
df.traffic_category.value_counts()
# Benign and background have significantly more data than other attack traffic.

In [ ]:
sns.countplot(y=df["traffic_category"])

^ Possible fix?
- Oversampling
- Undersampling
- Data augmentation

## 1. SMOTE All features

Decision tree model dtmodel using all traffic_category classes & oversampled to match the majority class (Benign traffic).  All features aswell.
Promising score.

In [ ]:
from collections import Counter
from sklearn.tree import DecisionTreeClassifier #Decision tree
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import SMOTE

In [ ]:
X=df.drop(['Label','traffic_category'], axis=1)
y=df.traffic_category

In [ ]:
counter = Counter(y)
print(counter)

In [ ]:
for label, _ in counter.items():
  row_ix = where(y == label)[0]
  pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

In [ ]:
oversample = SMOTE()
X_over, y_over = oversample.fit_resample(X_top, y)
counter= Counter(y_over)
print(counter)

In [ ]:
counter = Counter(y_over)
print(counter)

In [ ]:
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_over, y_over, test_size=0.3, random_state=1)

In [ ]:
dtSMOTE = DecisionTreeClassifier(random_state=1)

dtSMOTE = dtSMOTE.fit(X_train_o,y_train_o)

In [ ]:
y_test_o_pred=dtSMOTE.predict(X_test_o)

In [ ]:
from sklearn.metrics import classification_report
report_o = classification_report(y_test_o, y_test_o_pred)
print(report_o)

In [ ]:
# Confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

cm = confusion_matrix(y_test_o, y_test_o_pred, labels=dtSMOTE.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtSMOTE.classes_)
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax)
plt.show()


In [ ]:
df_dt = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred.flatten()})
df_dt.head(100)

In [ ]:
from google.colab import files
df_dt.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')

## 2. Undersampling All features

- Report score shows that benign accuracy and score drop from usual probably because of the lack of data. Background on the other hand still shows acceptable score. Changing test size ratio does not really change score.

In [ ]:
#train test

In [ ]:
X = df.drop(['Label','traffic_category'], axis=1)
y = df.traffic_category

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier #Decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from numpy import mean


#Summarize class distribution
print(Counter(y))

#Defining the undersample strategy
undersample = RandomUnderSampler(sampling_strategy = 'not minority')

#fit and apply
X_under, y_under = undersample.fit_resample(X_top,y)

#Summarize new class distribution
print(Counter(y_under))

#Another time to resample background aswell. may need to use alt code


In [ ]:
# Define pipeline
steps = [('under', RandomUnderSampler()),('model', DecisionTreeClassifier())]
pipeline_under = Pipeline(steps=steps)

# Eval pipeline

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores_u = cross_val_score(pipeline_under, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
score_u = mean(scores_u)
print('F1 score is: %.3f' % score_u)

In [ ]:
X_under.shape

In [ ]:
y_under.shape

In [ ]:
X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(X_under, y_under, test_size=0.3, random_state=1)

In [ ]:
dtRUS = DecisionTreeClassifier(random_state=1)

dtRUS = dtRUS.fit(X_train_u,y_train_u)

In [ ]:
y_train_u.value_counts(normalize=False).sort_index()

In [ ]:
y_test_u.value_counts(normalize=False).sort_index()

In [ ]:
y_test_u_pred = dtRUS.predict(X_test_u)

In [ ]:
y_test_u_pred = dtRUS.predict_proba(X_test_u)

In [ ]:
from sklearn.metrics import classification_report
report_u = classification_report(y_test_u, y_test_u_pred)
print(report_u)

In [ ]:
from sklearn.metrics import f1_score

# Calculating the F1 score of classifier
print(f"F1 Score of the classifier is: {f1_score(y_test_u, y_test_u_pred, average='macro')}")

In [ ]:
from sklearn.metrics import precision_score

# Calculating the precision score of classifier
print("Precision Score : ",precision_score(y_test_u, y_test_u_pred, pos_label='positive',average='weighted'))

In [ ]:
from sklearn.metrics import accuracy_score
# Calculating the accuracy of classifier
print(f"Accuracy of the classifier is: {accuracy_score(y_test_u, y_test_u_pred)}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

cm = confusion_matrix(y_test_u, y_test_u_pred, labels=dtRUS.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dtRUS.classes_)
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax)
plt.show()


## Imbalanced original Dataset variable.

In [ ]:
X=df.drop(['Label','traffic_category'], axis=1)
y=df.traffic_category

In [ ]:
top_col=['flow_pkts_payload.std','flow_iat.min','flow_pkts_payload.max','down_up_ratio','fwd_pkts_payload.max','fwd_iat.min','fwd_subflow_bytes','flow_iat.max','payload_bytes_per_second','active.tot','bwd_iat.tot','flow_iat.std','flow_pkts_payload.min','fwd_last_window_size','active.min','bwd_pkts_payload.min','fwd_iat.max','fwd_header_size_tot','bwd_iat.max','bwd_iat.std','bwd_init_window_size','bwd_iat.min','fwd_pkts_payload.std','fwd_iat.std','fwd_init_window_size','flow_duration','flow_duration','flow_iat.avg','flow_pkts_payload.tot','fwd_iat.tot','bwd_pkts_payload.std','bwd_iat.avg','fwd_pkts_per_sec','flow_pkts_payload.avg','active.max','fwd_pkts_payload.tot','active,avg','bwd_pkts_payload.avg','fwd_header_size_max','fwd_pkts_payload.avg']
X_top = df[[c for c in df.columns if c in top_col]]
y=df.traffic_category

## SMOTE Important features.



### FI using imbalanced Dataset
- Feature selection using various methods.

#### ANNOVA f_classif()

      'fwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
       'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_min',
       'fwd_header_size_max', 'bwd_header_size_min', 'bwd_header_size_max',
       'flow_FIN_flag_count', 'flow_SYN_flag_count', 'flow_RST_flag_count',
       'fwd_PSH_flag_count', 'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
       'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
       'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.avg',
       'bwd_pkts_payload.std', 'flow_pkts_payload.min',
       'flow_pkts_payload.max', 'flow_pkts_payload.avg',
       'flow_pkts_payload.std', 'bwd_iat.avg', 'fwd_subflow_pkts',
       'bwd_subflow_pkts', 'fwd_subflow_bytes', 'bwd_bulk_packets',
       'bwd_bulk_rate', 'active.min', 'fwd_init_window_size',
       'bwd_init_window_size', 'fwd_last_window_size'],
      dtype='object')

Half features 36

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X = df.drop(['Label','traffic_category'], axis=1)
y = df.traffic_category

In [ ]:
len(X.columns)

In [ ]:
# Define feature selection
fs1 = SelectKBest(score_func =f_classif, k=36) #k is number features you want
# Apply feature selection
fs1.fit_transform(X, y)

In [ ]:
# Get columns to keep and create new dataframe with those only
cols_idxs = fs1.get_support(indices=True)
X_new = X.iloc[:,cols_idxs]

In [ ]:
X_new.head(2)

In [ ]:
X_new.shape

In [ ]:
def f(var):
    if isinstance(var, pd.DataFrame):
        print("yes lol")

f(X_new)

In [ ]:
X_new.columns

#### **Oversampling the selected features from Imbalanced dataset.**

The 36 features selected but from an imbalanced so it may include features that are not actually important.

In [ ]:
from collections import Counter
from sklearn.tree import DecisionTreeClassifier #Decision tree
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import SMOTE

In [ ]:
counter = Counter(y)
print(counter)

In [ ]:
oversample = SMOTE()
X_over, y_over = oversample.fit_resample(X_new, y)
counter= Counter(y_over)
print(counter)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.3, random_state=1)

In [ ]:
# model using selected features from Imbalanced Dataset
dtmodel_fs_ID = DecisionTreeClassifier(random_state=1)

dtmodel_fs_ID = dtmodel_fs_ID.fit(X_train,y_train)

In [ ]:
y_test_pred=dtmodel_fs_ID.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
report_OSFID = classification_report(y_test, y_test_pred)
print(report_OSFID)

### FI using OVERSAMPLED DATASET

Oversampled the data, and then conduct feature selection.

In [ ]:
from collections import Counter
from sklearn.tree import DecisionTreeClassifier #Decision tree
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import SMOTE

In [ ]:
counter = Counter(y)
print(counter)

In [ ]:
X.shape

In [ ]:
# Oversampling the dataset
oversample = SMOTE()
X_over2, y_over2 = oversample.fit_resample(X, y)
counter= Counter(y_over2)
print(counter)

In [ ]:
fs2 = SelectKBest(score_func =f_classif, k=36) #k is number features you want
# Apply feature selection
fs2.fit_transform(X_over2, y_over2)

In [ ]:
# Get new features with oversampled data
cols_name = fs2.get_support(indices=True)
X_new_over= X_over2.iloc[:,cols_name]

In [ ]:
X_new_over.shape

In [ ]:
y_over2.shape

In [ ]:
X_new_over.head(1)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_new_over, y_over2, test_size=0.3, random_state=1)

In [ ]:
dtmodel_fs_OverD = DecisionTreeClassifier(random_state=1)

dtmodel_fs_OverD = dtmodel_fs_OverD.fit(X_train2,y_train2)

In [ ]:
y_test_pred_2=dtmodel_fs_OverD.predict(X_test2)

In [ ]:
from sklearn.metrics import classification_report
report_OSFOverD = classification_report(y_test2, y_test_pred_2)
print(report_OSFOverD)

## Section : Using Benign - Background - 1 Attack Traffic?
Testing if we can achieve result in better model performance in recognizing traffic type without interference from other attack that may have similar feature values.

In [ ]:
# New dataframe with only benign, background, and 1 attack traffic (Bruteforce)
df_brutef = df[ (df['traffic_category'] != "Bruteforce-XML") & (df['traffic_category'] != "Probing") & (df['traffic_category'] != "XMRIGCC CryptoMiner")]
df_brutef.traffic_category.value_counts()

### Undersampling Dataset

Decreasing the majority data classes (Benign and background) to a similar amount to the minority bruteforce data.

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier #Decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from numpy import mean

X = df_brutef.drop(['Label','traffic_category'], axis=1)
y = df_brutef.traffic_category

#Summarize class distribution
print(Counter(y))

#Defining the undersample strategy
undersample = RandomUnderSampler(sampling_strategy = 'not minority')

#fit and apply
X_under, y_under = undersample.fit_resample(X,y)

#Summarize new class distribution
print(Counter(y_under))

#Another time to resample background aswell. may need to use alt code
X_under, y_under = undersample.fit_resample(X_under,y_under)
print(Counter(y_under))

In [ ]:
X_under.head(3)

In [ ]:
# Define pipeline
steps = [('under', RandomUnderSampler()),('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)

# Eval pipeline

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='f1_micro', cv=cv, n_jobs=-1)
score = mean(scores)
print('F1 score is: %.3f' % score)

### Oversampling Dataset
Increasing minority data (bruteforce) to be the same with the majority.

In [ ]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier #Decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from numpy import mean

print(Counter(y))

#Defining the oversample strategy
oversample = RandomOverSampler(sampling_strategy = 'minority')

#fit and apply
X_over, y_over = oversample.fit_resample(X,y)

#Summarize new class distribution
print(Counter(y_over))

#Another time to resample background aswell. may need to use alt code
X_over, y_over = oversample.fit_resample(X_over,y_over)
print(Counter(y_over))

In [ ]:
steps = [('over', RandomOverSampler()),('model', DecisionTreeClassifier())]
pipeline2 = Pipeline(steps=steps)

# Eval pipeline

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline2, X, y, scoring='f1_micro', cv=cv, n_jobs=-1)
score = mean(scores)
print('F1 score is: %.3f' % score)

### DT using undersampled df_brutef

In [ ]:
X_under.shape

In [ ]:
y_under.shape

In [ ]:
from sklearn.model_selection import train_test_split

#training and testing split using all feature
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.3, random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtmodel = DecisionTreeClassifier(random_state=1)

dtmodel = dtmodel.fit(X_train,y_train)

In [ ]:
y_test_pred=dtmodel.predict(X_test)

In [ ]:
y_train.value_counts(normalize=True).sort_index()

In [ ]:
y_test.value_counts(normalize=True).sort_index()

#Same ratio of distribution in test (and train set)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_test_pred)
print(report)

### DT using oversampled df_brutef

In [ ]:
X_over.shape

In [ ]:
y_over.shape

In [ ]:
from sklearn.model_selection import train_test_split

#training and testing split using all feature
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_over, y_over, test_size=0.3, random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtmodelOS = DecisionTreeClassifier(random_state=1)

dtmodelOS = dtmodelOS.fit(X_train2,y_train2)

In [ ]:
y_test_pred2=dtmodel.predict(X_test2)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test2, y_test_pred2)
print(report)

### Testing **dtmodel** undersampled data using original imbalanced dataset.

Report shows better result for Bruteforce compared to model using imbalanced original dataset with not sampling. Although result maybe slighty worse because of imbalanced original dataset and other attack traffic.

In [ ]:
df.traffic_category.value_counts()

In [ ]:
X_N = df.drop(['Label','traffic_category'], axis=1)
y_N = df.traffic_category

In [ ]:
X_trainN, X_testN, y_trainN, y_testN = train_test_split(X_N, y_N, test_size=0.3, random_state=1, stratify=y_N)

In [ ]:
dtmodelN = DecisionTreeClassifier(random_state=1)

dtmodelN = dtmodel.fit(X_trainN,y_trainN)

In [ ]:
y_test_predN=dtmodel.predict(X_testN)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_testN, y_test_predN)
print(report)